# Passer Ratings: Hand Sizes

Hypothesis: Hand size affects average seasonal passer rating.

QB's are apparently meant to have hands above 9.25 inches.

In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import sys
sys.path.append('..')

In [2]:
cnx = mysql.connector.connect(user='root', password='mOntie20!mysql', host='127.0.0.1', database='nfl')
sql = open('../sql/qb_season_ratings.sql', 'r').read()
df = pd.read_sql(sql, cnx)
df = df.loc[:,~df.columns.duplicated()][df['attempts']>50]

Theres a lot of missing hand measurements so we'll have to do some Googling.

In [3]:
df['hand'].replace([0,''], np.nan, inplace=True)
df['height'].replace([0,''], np.nan, inplace=True)
df['weight'].replace([0,''], np.nan, inplace=True)

In [4]:
df['hand'] = df.groupby('player')['hand'].apply(lambda x: x.fillna(method='ffill'))

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1014 entries, 0 to 1013
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   player         1014 non-null   object 
 1   name           1014 non-null   object 
 2   first_name     1014 non-null   object 
 3   last_name      1014 non-null   object 
 4   year           1014 non-null   int64  
 5   attempts       1014 non-null   float64
 6   completions    1014 non-null   float64
 7   yards          1014 non-null   float64
 8   interceptions  1014 non-null   float64
 9   TD             1014 non-null   float64
 10  avg_rating     1014 non-null   float64
 11  height         1014 non-null   int64  
 12  weight         1014 non-null   int64  
 13  hand           515 non-null    float64
 14  arm            1014 non-null   float64
 15  draft          1014 non-null   int64  
 16  start          1014 non-null   int64  
 17  forty          1014 non-null   float64
 18  bench   

In [6]:
missing_hands = df[df['hand'].isna()].groupby(['player','first_name','last_name']).size().sort_values(ascending=False)

In [7]:
missing = missing_hands.to_frame().reset_index().set_index('player')
missing['size'] = np.nan
missing.head()

,first_name,last_name,0,size
player,,,,
TB-2300,Tom,Brady,18,NaN
PM-0200,Peyton,Manning,15,NaN
RF-0500,Ryan,Fitzpatrick,13,NaN
MH-1600,Matt,Hasselbeck,13,NaN
DM-2200,Donovan,McNabb,12,NaN


We'll use Selenium to scrape hand sizes from the web.

In [8]:
from scripts.qb_bot import QBBot

bot = QBBot()

for i, x in missing.iterrows():
    missing.loc[i,'size'] = bot.search(x['first_name'],x['last_name'])
    break

Tom Brady 9.38


In [9]:
missing

,first_name,last_name,0,size
player,,,,
TB-2300,Tom,Brady,18,9.38
PM-0200,Peyton,Manning,15,NaN
RF-0500,Ryan,Fitzpatrick,13,NaN
MH-1600,Matt,Hasselbeck,13,NaN
DM-2200,Donovan,McNabb,12,NaN
...,...,...,...,...
DW-5000,Danny,Wuerffel,1,NaN
DP-0800,Doug,Pederson,1,NaN
TC-2300,Todd,Collins,1,NaN


In [10]:
missing.to_csv('missing_hands_scraped.csv')